****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [359]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [360]:
json_df = pd.read_json('data/world_bank_projects.json')

In [361]:
#the 10 countries with most projects:
json_df['countrycode'].value_counts().head(10)

CN    19
ID    19
VN    17
IN    16
RY    13
NP    12
BD    12
MA    12
MZ    11
3A    11
Name: countrycode, dtype: int64

In [362]:
#Find the top 10 major project themes (using column 'mjtheme_namecode')
my_list=[]
for my_item in json_df['mjtheme_namecode']:
    my_list.append(my_item)

In [363]:
my_list[:11]

[[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}],
 [{'code': '1', 'name': 'Economic management'},
  {'code': '6', 'name': 'Social protection and risk management'}],
 [{'code': '5', 'name': 'Trade and integration'},
  {'code': '2', 'name': 'Public sector governance'},
  {'code': '11', 'name': 'Environment and natural resources management'},
  {'code': '6', 'name': 'Social protection and risk management'}],
 [{'code': '7', 'name': 'Social dev/gender/inclusion'},
  {'code': '7', 'name': 'Social dev/gender/inclusion'}],
 [{'code': '5', 'name': 'Trade and integration'},
  {'code': '4', 'name': 'Financial and private sector development'}],
 [{'code': '6', 'name': 'Social protection and risk management'},
  {'code': '6', 'name': ''}],
 [{'code': '2', 'name': 'Public sector governance'},
  {'code': '4', 'name': 'Financial and private sector development'}],
 [{'code': '11', 'name': 'Environment and natural resources management'},
  {'code': '8', 'name': ''}],
 [{'code':

In [364]:
# old code:
#new_list=[]
#for my_item in my_list:
#    for my_dict in my_item:
#        new_list.append(my_dict)

In [365]:
# new code
# get a flat list of dictionaries
very_new_list = [item for sublist in my_list for item in sublist]

In [366]:
very_new_list[:15]

[{'code': '8', 'name': 'Human development'},
 {'code': '11', 'name': ''},
 {'code': '1', 'name': 'Economic management'},
 {'code': '6', 'name': 'Social protection and risk management'},
 {'code': '5', 'name': 'Trade and integration'},
 {'code': '2', 'name': 'Public sector governance'},
 {'code': '11', 'name': 'Environment and natural resources management'},
 {'code': '6', 'name': 'Social protection and risk management'},
 {'code': '7', 'name': 'Social dev/gender/inclusion'},
 {'code': '7', 'name': 'Social dev/gender/inclusion'},
 {'code': '5', 'name': 'Trade and integration'},
 {'code': '4', 'name': 'Financial and private sector development'},
 {'code': '6', 'name': 'Social protection and risk management'},
 {'code': '6', 'name': ''},
 {'code': '2', 'name': 'Public sector governance'}]

In [367]:
from collections import defaultdict
my_dict={}
for item in very_new_list:
    if not item.get('name') in my_dict:
        my_dict[item.get('name')]=0
    my_dict[item.get('name')] += 1

print(sorted(((value,key) for (key,value) in my_dict.items()), reverse=True)[:10])

[(223, 'Environment and natural resources management'), (202, 'Rural development'), (197, 'Human development'), (184, 'Public sector governance'), (158, 'Social protection and risk management'), (130, 'Financial and private sector development'), (122, ''), (119, 'Social dev/gender/inclusion'), (72, 'Trade and integration'), (47, 'Urban development')]


In 2. above you will notice that some entries have only the code and the name is missing.
Create a dataframe with the missing names filled in.

The new dataframe is json_copy_df

In [368]:
json_copy_df = json_df.copy()

In [369]:
# get the correct values for each code, ignore null strings
# use original dataframe, with null strings
small_dict={}
for item in very_new_list:
    if len(item.get('name'))>0:
        small_dict[item.get('code')]=item.get('name')

In [370]:
small_dict

{'8': 'Human development',
 '1': 'Economic management',
 '6': 'Social protection and risk management',
 '5': 'Trade and integration',
 '2': 'Public sector governance',
 '11': 'Environment and natural resources management',
 '7': 'Social dev/gender/inclusion',
 '4': 'Financial and private sector development',
 '10': 'Rural development',
 '9': 'Urban development',
 '3': 'Rule of law'}

In [371]:
# find a null value and check how it is modified later on
json_df['mjtheme_namecode'][10]

[{'code': '10', 'name': 'Rural development'}, {'code': '2', 'name': ''}]

In [372]:
for one_more_list in json_copy_df['mjtheme_namecode']:
    for second_dict in one_more_list:
        if len(second_dict.get('name'))==0:
            second_dict['name']=small_dict[second_dict.get('code')]

In [373]:
# it looks like everything is ok!
json_df['mjtheme_namecode'][10]

[{'code': '10', 'name': 'Rural development'},
 {'code': '2', 'name': 'Public sector governance'}]

However, the list for the top 10 major project themes included a '' project (before processing the copied dataframe)
There is one more thing to do, to run again the count for the 10 projects and check id the '' projects are gone

In [374]:
my_list=[]
for my_item in json_copy_df['mjtheme_namecode']:
    my_list.append(my_item)
    
very_new_list = [item for sublist in my_list for item in sublist]
    
my_dict={}
for item in very_new_list:
    if not item.get('name') in my_dict:
        my_dict[item.get('name')]=0
    my_dict[item.get('name')] += 1

print(sorted(((value,key) for (key,value) in my_dict.items()), reverse=True)[:10])

[(250, 'Environment and natural resources management'), (216, 'Rural development'), (210, 'Human development'), (199, 'Public sector governance'), (168, 'Social protection and risk management'), (146, 'Financial and private sector development'), (130, 'Social dev/gender/inclusion'), (77, 'Trade and integration'), (50, 'Urban development'), (38, 'Economic management')]


PS I did not understand how to properly use json_normalize so I have chosen the hard way... I am sure that the solution is very simple. I'd like to see the simple solution!
Thanks